In [2]:
#Note: before launching Jupyter lab, need to export: export DJANGO_SETTINGS_MODULE=crypto_signal.settings
import django
django.setup()

In [3]:
from crypto_track.models import CryptoCandle, PyTrends, Bank, SignalSimulation, Simulation
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [16]:
# attach next day's close price so we can estimate
candle_data = list(CryptoCandle.objects.all().values().order_by('-period_start_timestamp'))
next_candle = ""
for i, cd in enumerate(candle_data):
    candle_data[i]['next_candle_period_close'] = next_candle
    next_candle = candle_data[i]['period_close']

In [17]:
print(candle_data[0])

{'id': 21226, 'crypto_traded': 'BTC', 'currency_quoted': 'USD', 'period_interval': '1d', 'period_start_timestamp': '2018-11-20T00:00:00Z', 'search_trend_id': None, 'period_low': Decimal('4120.4568100000'), 'period_open': Decimal('4813.8118400000'), 'period_close': Decimal('4746.7833500000'), 'period_high': Decimal('5225.4696500000'), 'period_volume': Decimal('1847523760.0000000000'), 'data_source': 'Nomics https://api.nomics.com/v1/candles', 'update_timestamp': datetime.datetime(2018, 11, 20, 18, 21, 37, 336293, tzinfo=<UTC>), 'next_candle_period_close': ''}


In [18]:
# sort by start date ascending
candle_data.sort(key=lambda item: item['period_start_timestamp'])

In [19]:
print(candle_data[0])

{'id': 19090, 'crypto_traded': 'BTC', 'currency_quoted': 'USD', 'period_interval': '1d', 'period_start_timestamp': '2013-01-14T00:00:00Z', 'search_trend_id': datetime.date(2013, 1, 14), 'period_low': Decimal('14.5329500000'), 'period_open': Decimal('14.5373700000'), 'period_close': Decimal('14.5329500000'), 'period_high': Decimal('14.5373700000'), 'period_volume': Decimal('295.0000000000'), 'data_source': 'Nomics https://api.nomics.com/v1/candles', 'update_timestamp': datetime.datetime(2018, 11, 20, 18, 21, 37, 336293, tzinfo=<UTC>), 'next_candle_period_close': Decimal('14.7685700000')}


In [20]:
df = pd.DataFrame(candle_data)
df.head()

,crypto_traded,currency_quoted,data_source,id,next_candle_period_close,period_close,period_high,period_interval,period_low,period_open,period_start_timestamp,period_volume,search_trend_id,update_timestamp
0,BTC,USD,Nomics https://api.nomics.com/v1/candles,19090,14.7685700000,14.5329500000,14.5373700000,1d,14.5329500000,14.5373700000,2013-01-14T00:00:00Z,295.0000000000,2013-01-14,2018-11-20 18:21:37.336293+00:00
1,BTC,USD,Nomics https://api.nomics.com/v1/candles,19091,15.0246000000,14.7685700000,14.7685700000,1d,14.5247900000,14.5247900000,2013-01-15T00:00:00Z,945.0000000000,2013-01-15,2018-11-20 18:21:37.336293+00:00
2,BTC,USD,Nomics https://api.nomics.com/v1/candles,19092,15.9646800000,15.0246000000,15.0246000000,1d,14.7492000000,14.7492000000,2013-01-16T00:00:00Z,2419.0000000000,2013-01-16,2018-11-20 18:21:37.336293+00:00
3,BTC,USD,Nomics https://api.nomics.com/v1/candles,19093,15.9980000000,15.9646800000,15.9646800000,1d,15.1367900000,15.1367900000,2013-01-17T00:00:00Z,8011.0000000000,2013-01-17,2018-11-20 18:21:37.336293+00:00
4,BTC,USD,Nomics https://api.nomics.com/v1/candles,19094,15.9273000000,15.9980000000,16.2627800000,1d,15.7488000000,16.0343700000,2013-01-18T00:00:00Z,5762.0000000000,2013-01-18,2018-11-20 18:21:37.336293+00:00


In [24]:
df_convert = df.replace( '',np.nan, regex=True)
df['period_close'] = df.period_close.astype(float)
df = df[(len(df.next_candle_period_close) > 0)]
#df['next_candle_period_close'] = df.next_candle_period_close.astype(float)

df.dtypes

KeyError: True

In [21]:
import numpy as np
df = df[(df.next_candle_period_close > 0)]


TypeError: '>' not supported between instances of 'str' and 'int'

In [21]:
# Create target object and call it y
y = df.next_candle_period_close

In [22]:
features = ['period_close', 'period_high', 'period_low', 'period_open', 'period_volume']
X = df[features]

In [24]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [25]:
# Specify Model
btc_model = DecisionTreeRegressor(random_state=1)
# Fit Model
btc_model.fit(train_X, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [26]:
# Make validation predictions and calculate mean absolute error
val_predictions = btc_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

ValueError: could not convert string to float: 